In [1]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json

# Adding for query to find top ten stations 
#from pymongo.collection import Collection
#from pymongo.aggregation import Aggregation

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# check our list of databases
print(mongo.list_database_names())

['admin', 'autosaurus', 'chicago_bikes', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'travel_db', 'uk_food']


In [4]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['withLatLong', 'RouteDistance', 'divvy_ridedata', 'Top10EndStations', 'weather_daily', 'withoutStationName', 'divvy_rides_by_season', 'divvy_rides_by_month', 'Top10StartStations', 'withStationName', 'divvy_ridedata_merged', 'Top10Routes']


In [6]:
divvy_rides = db['divvy_ridedata']
weather_daily = db['weather_daily']
withStation = db['withStationName']
withoutStation = db['withoutStationName']
divvy_ridedata_merged = db['divvy_ridedata_merged']

In [7]:
# Create a pipeline query to find docouments that have lat/long  
pipeline = [
    {
        "$match": {
            "$and": [
                { "start_lat": { "$ne": "" } },
                { "start_lat": { "$ne": 0 } },
                { "start_lng": { "$ne": "" } },
                { "start_lng": { "$ne": 0 } },
                { "end_lat": { "$ne": "" } },
                { "end_lat": { "$ne": 0 } },
                { "end_lng": { "$ne": "" } },
                { "end_lng": { "$ne": 0 } }
            ]
        }
    }, 
    {"$out": "withLatLong"}
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))

In [8]:
# review the collections in our new database
print(db.list_collection_names())

['RouteDistance', 'divvy_ridedata', 'Top10EndStations', 'withLatLong', 'weather_daily', 'withoutStationName', 'divvy_rides_by_season', 'divvy_rides_by_month', 'Top10StartStations', 'withStationName', 'divvy_ridedata_merged', 'Top10Routes']


In [9]:
#Assign collection to a variable
withLatLong = db['withLatLong']

In [10]:
print(withLatLong.find_one())

{'_id': ObjectId('64e4eab4a682807a5695d165'), 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'end_station_id': 'RP-007', 'end_station_name': 'Clark St & Touhy Ave', 'ended_at': '2022-01-13 12:02:44', 'ended_at_date': '2022-01-13', 'ended_at_time': '12:02:44', 'member_casual': 'casual', 'ride_id': 'C2F7DD78E82EC875', 'rideable_type': 'electric_bike', 'start_lat': 42.0128005, 'start_lng': -87.665906, 'start_station_id': 525, 'start_station_name': 'Glenwood Ave & Touhy Ave', 'started_at': '2022-01-13 11:59:47', 'started_at_date': '2022-01-13', 'started_at_time': '11:59:47', 'weather_data': {'_id': ObjectId('64e4ebecb589d1838b565c57'), 'date': '2022-01-13', 'cloud_cover': 75.0, 'precipitation': 0.0, 'min_temp': 30.72, 'max_temp': 39.83, 'morning_temp': 35.49, 'afternoon_temp': 30.72, 'evening_temp': 34.38, 'night_temp': 39.22, 'max_windspeed': 8.01}}


In [11]:
# Create a pipeline query to find distance of each route in descending order by length  
pipeline = [
    {
        "$addFields": {
            "start_lat": { "$toDouble": "$start_lat" },
            "start_lng": { "$toDouble": "$start_lng" },
            "end_lat": { "$toDouble": "$end_lat" },
            "end_lng": { "$toDouble": "$end_lng" }
        }
    },
    {
        "$addFields": {
            "distance": {
                "$sqrt": {
                    "$add": [
                        {
                            "$pow": [
                                { "$subtract": ["$end_lat", "$start_lat"] },
                                2
                            ]
                        },
                        {
                            "$pow": [
                                {
                                    "$multiply": [
                                        { "$subtract": ["$end_lng", "$start_lng"] },
                                        { "$cos": { "$avg": ["$start_lat", "$end_lat"] } }
                                    ]
                                },
                                2
                            ]
                        }
                    ]
                }
            }
        }
    },
    {
        "$sort": {"distance": -1}
    },
    {"$out": "RouteDistance"}
]

# Perform the aggregation
result = list(withLatLong.aggregate(pipeline))

# Assign to a variable
RouteDistance = db['RouteDistance']

In [12]:
# review the collections in our new database
print(db.list_collection_names())

['divvy_ridedata', 'Top10EndStations', 'withLatLong', 'weather_daily', 'withoutStationName', 'divvy_rides_by_season', 'divvy_rides_by_month', 'Top10StartStations', 'withStationName', 'RouteDistance', 'divvy_ridedata_merged', 'Top10Routes']


In [13]:
RouteDistance = db['RouteDistance']

In [14]:
print(RouteDistance.find_one())

{'_id': ObjectId('64e4eab4a682807a5695ff56'), 'end_lat': 41.8646, 'end_lng': -87.681, 'end_station_id': 'Pawel Bialowas - Test- PBSC charging station', 'end_station_name': 'Pawel Bialowas - Test- PBSC charging station', 'ended_at': '2022-01-14 11:15:50', 'ended_at_date': '2022-01-14', 'ended_at_time': '11:15:50', 'member_casual': 'casual', 'ride_id': '3327172413547F64', 'rideable_type': 'electric_bike', 'start_lat': 45.635034323, 'start_lng': -73.79647696, 'start_station_id': 'Pawel Bialowas - Test- PBSC charging station', 'start_station_name': 'Pawel Bialowas - Test- PBSC charging station', 'started_at': '2022-01-14 11:13:15', 'started_at_date': '2022-01-14', 'started_at_time': '11:13:15', 'weather_data': {'_id': ObjectId('64e4ebecb589d1838b565c58'), 'date': '2022-01-14', 'cloud_cover': 34.0, 'precipitation': 0.0, 'min_temp': 30.38, 'max_temp': 38.8, 'morning_temp': 35.74, 'afternoon_temp': 35.15, 'evening_temp': 32.5, 'night_temp': 38.39, 'max_windspeed': 10.0}, 'distance': 14.027235